In [2]:
from vnstock import *
from datetime import date, datetime
import pandas as pd


**Vui lòng chuyển đổi sang Vnstock3** thế hệ mới (3.2.1) với câu lệnh: `pip install vnstock3 --upgrade`.
**Từ 1/1/2025, vnstock3 sẽ được cài đặt khi sử dụng cú pháp** `pip install vnstock` **thay cho Vnstock Legacy** hiện tại.
Xem chi tiết [chuyển đổi sang vnstock3](https://vnstocks.com/docs/tai-lieu/migration-chuyen-doi-sang-vnstock3).
Phiên bản **Vnstock Legacy (0.2.9.2.3)** bạn đang sử dụng **sẽ không được nâng cấp thêm.**
Từ 7/10/2024 Vnstock giới thiệu nhóm Facebook Cộng đồng Vnstock, tham gia thảo luận tại đây: https://www.facebook.com/groups/vnstock.official

In [3]:
# Hàm lấy giá cố phiếu
def get_stock_price(stock):
    stock_hist = stock_historical_data(stock, "2022-10-01",
        str("2024-11-03"), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['time'] = stock_hist['time'].astype(str)
    stock_hist = stock_hist.rename(columns={'ticker': 'stock_name', 'time': 'date'})

    return stock_hist
get_stock_price('HSG')


,date,open,high,low,close,volume,stock_name,%_change,change
0,2022-10-03,13600,13650,12690,12690,8849000,HSG,NaN,NaN
1,2022-10-04,12860,12960,12000,12000,6779700,HSG,-5.44,-690.0
2,2022-10-05,12370,12590,12140,12460,4890900,HSG,3.83,460.0
3,2022-10-06,12460,12540,11630,11630,8350300,HSG,-6.66,-830.0
4,2022-10-07,11120,11400,10850,10850,10420600,HSG,-6.71,-780.0
...,...,...,...,...,...,...,...,...,...
518,2024-10-28,20060,20310,19920,20160,6509300,HSG,2.23,440.0
519,2024-10-29,20310,20310,20010,20110,5104500,HSG,-0.25,-50.0
520,2024-10-30,20110,20160,19720,19770,5431900,HSG,-1.69,-340.0
521,2024-10-31,19720,19820,19630,19670,5818700,HSG,-0.51,-100.0


In [ ]:
def get_stock_fundamental_info(stock):
    fundamental_ratio = company_fundamental_ratio(stock) 
    profile = company_profile(stock)
    overview = company_overview(stock)
    evaluation = stock_evaluation(stock)[["ticker", "PE", "industryPE", "PB", "industryPB"]][-1:]
    stock_info = pd.merge(fundamental_ratio[['ticker']], profile[['companyName', 'ticker']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, overview[['ticker', 'outstandingShare', 'exchange']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, evaluation, on='ticker', how='inner')

    stock_info = stock_info.rename(columns={'ticker': 'stock_name', 'companyName': 'company_name', 'outstandingShare': 'outstanding_share'})
    stock_info['outstanding_share'] = stock_info['outstanding_share'] * 1000000

    return stock_info

In [31]:
get_stock_fundamental_info('VIC')

    ticker   fromDate     toDate    PE   PB  industryPE  vnindexPE  \
0      VIC 2024-04-17 2024-04-17  48.0  1.3        15.8       14.8   
1      VIC 2024-04-19 2024-04-19  45.4  1.3        15.3       14.5   
2      VIC 2024-04-22 2024-04-22  44.8  1.2        15.5       14.8   
3      VIC 2024-04-23 2024-04-23  43.7  1.2        15.1       14.5   
4      VIC 2024-04-24 2024-04-24  44.3  1.2        15.5       14.8   
..     ...        ...        ...   ...  ...         ...        ...   
243    VIC 2025-04-10 2025-04-10  19.6  1.7        17.0       12.9   
244    VIC 2025-04-11 2025-04-11  20.9  1.8        17.6       13.4   
245    VIC 2025-04-14 2025-04-14  22.4  1.9        18.2       13.5   
246    VIC 2025-04-15 2025-04-15  22.6  2.0        18.1       13.3   
247    VIC 2025-04-16 2025-04-16  21.8  1.9        17.6       13.2   

     industryPB  vnindexPB  
0           1.2        1.7  
1           1.2        1.6  
2           1.2        1.7  
3           1.1        1.6  
4           1.

,stock_name,company_name,outstanding_share,exchange,PE,industryPE,PB,industryPB
0,VIC,Tập đoàn Vingroup - Công ty CP,3.823700e+09,HOSE,21.8,17.6,1.9,1.2


### Tổng hợp tất cả các DataFrame

In [40]:
Stocks = ['CTG','HSG','CMG','VIC']
Stock_price_list = []
Stock_info_list = []
for stock in Stocks:
    stock_info = get_stock_fundamental_info(stock)
    stock_price = get_stock_price(stock)

    Stock_price_list.append(stock_price)
    Stock_info_list.append(stock_info)

Stock_price_df = pd.concat(Stock_price_list,ignore_index=True)
Stock_info_df = pd.concat(Stock_info_list,ignore_index=True)

In [41]:
Stock_price_df

,date,open,high,low,close,volume,stock_name,%_change,change
0,2022-10-03,20310,20580,19330,19330,5690400,CTG,NaN,NaN
1,2022-10-04,19690,20310,18350,18570,5635700,CTG,-3.93,-760.0
2,2022-10-05,19060,19870,19060,19690,4275300,CTG,6.03,1120.0
3,2022-10-06,19690,19910,19020,19150,4256200,CTG,-2.74,-540.0
4,2022-10-07,19060,19060,17850,17850,8381600,CTG,-6.79,-1300.0
...,...,...,...,...,...,...,...,...,...
2092,2024-10-29,41500,41700,41300,41350,1285000,VIC,-0.24,-100.0
2093,2024-10-30,41450,41700,40800,41000,2024200,VIC,-0.85,-350.0
2094,2024-10-31,41400,41700,41050,41550,1827900,VIC,1.34,550.0
2095,2024-11-01,41550,41600,41100,41200,1338600,VIC,-0.84,-350.0


In [42]:
Stock_info_df

,stock_name,company_name,outstanding_share,exchange,PE,industryPE,PB,industryPB
0,CTG,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,5.370000e+09,HOSE,8.8,9.4,1.4,1.5
1,HSG,Công ty Cổ phần Tập đoàn Hoa Sen,6.210000e+08,HOSE,24.3,19.3,1.1,1.3
2,CMG,Công ty Cổ phần Tập đoàn Công nghệ CMC,1.904000e+08,HOSE,30.7,26.6,3.6,4.2
3,VIC,Tập đoàn Vingroup - Công ty CP,3.823700e+09,HOSE,16.5,18.1,1.2,1.1


In [43]:
Stock_price_df.to_csv(r'C:\Users\hsvie\Desktop\do_an\Data\Stock_price.csv')
Stock_info_df.to_csv(r'C:\Users\hsvie\Desktop\do_an\Data\Stock_info.csv')